# **DocuPy**
### _"Automate Documentation, Comments, and Unit Tests for Python Code"_


In [3]:
!pip install -q openai anthropic python-dotenv gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.2/208.2 kB 7.9 MB/s eta 0:00:00


## Setup and Install Dependencies

- Start by installing all necessary libraries.

In [8]:
# imports
import os
import io
import sys
import subprocess
import openai
import anthropic
import google.generativeai as gemini
import gradio as gr
from openai import OpenAI
from google.colab import userdata

## Add Secrets to the Colab Notebook

- Add the API keys for OpenAI, Claude, and Gemini to authenticate and access their respective models and services.


In [9]:
# Set your API keys
gpt = openai.OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
claude = anthropic.Anthropic(api_key=userdata.get('ANTHROPIC_API_KEY'))
gemini.configure(api_key=userdata.get('GOOGLE_API_KEY'))

## Define Required Constants

- Set up the necessary constants that will be used throughout the application.


In [38]:
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"
GEMINI_MODEL = "gemini-1.5-pro"

PI = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

## Create the User Interface (UI) with Gradio

- Design an intuitive and user-friendly Gradio interface.
- Ensure the UI is simple, responsive, and accessible for seamless interaction.




In [103]:
css = """
.python { background-color: #306998; }
.cpp { background-color: #050; }
.button { height: 4em; font-size: 1.5em; }
"""

In [104]:
# Tab to Document Code with Docstrings and Comments
def docs_comments_ui():
    with gr.Tab("Docstrings & Comments"):
        gr.Markdown("## Document Code with Docstrings and Comments")
        with gr.Row():
            python = gr.Textbox(label="Python Code:", value=PI, lines=20)
            python_with_comments = gr.TextArea(label="Python Code with Docstrings and Comments:", lines=20, elem_classes=["python"])
        with gr.Row():
            model = gr.Dropdown(["GPT", "Claude", "Gemini"], label="Select Model", value="GPT")
            comments_btn = gr.Button("Add Docstrings & Comments", elem_classes=["button"])

        comments_btn.click(lambda python, model: "DOCSTRINGS AND COMMENTS", inputs=[python, model], outputs=[python_with_comments])


In [108]:
# Tab to Generate Comprehensive Unit Tests
def unit_tests_ui():
    with gr.Tab("Unit Tests"):
        gr.Markdown("## Generate Comprehensive Unit Tests")
        with gr.Row():
            python = gr.Textbox(label="Python Ccode:", value=PI, lines=20)
            python_unit_tests = gr.TextArea(label="Python Code with Unit Tests:", lines=20, elem_classes=["python"])
        with gr.Row():
            model = gr.Dropdown(["GPT", "Claude", "Gemini"], label="Select Model", value="GPT")
            unit_tests_btn = gr.Button("Generate Unit Tests", elem_classes=["button"])

        unit_tests_btn.click(lambda python, model: "UNIT TESTS", inputs=[python, model], outputs=[python_unit_tests])


In [111]:
# Tab to Convert Python Code to C++
def python_to_cpp_ui():
    with gr.Tab("Python to C++"):
        gr.Markdown("## Convert Python Code to C++")
        with gr.Row():
            python = gr.Textbox(label="Python code:", value=PI, lines=20)
            cpp = gr.Textbox(label="C++ code:", lines=20)
        with gr.Row():
            model = gr.Dropdown(["GPT", "Claude", "Gemini"], label="Select Model", value="GPT")
            convert_btn = gr.Button("Convert to C++", elem_classes=["button"])
        with gr.Row():
            python_run = gr.Button("Run Python")
            cpp_run = gr.Button("Run C++")
        with gr.Row():
            python_out = gr.TextArea(label="Python result:", lines=10, elem_classes=["python"])  # Apply the 'python' CSS class for custom styling
            cpp_out = gr.TextArea(label="C++ result:", lines=10, elem_classes=["cpp"])  # Apply the 'cpp' CSS class for custom styling

        convert_btn.click(lambda python, model: "C++ CODE", inputs=[python, model], outputs=[cpp])
        python_run.click(lambda python: "PYTHON RUN", inputs=[python], outputs=[python_out])
        cpp_run.click(lambda cpp: "C++ RUN", inputs=[cpp], outputs=[cpp_out])

In [112]:
# Combine the tabs into the main UI
with gr.Blocks(css=css) as main_ui:
    docs_comments_ui()
    unit_tests_ui()
    python_to_cpp_ui()

# Launch the app
main_ui.launch(inbrowser=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2c9a8d60c936e1a2ca.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
